# VND Remote Python Connection

In this chapter you will learn to use a live Python text interface connected to a running VND session.   You will use the VND selection language to create Python neuron selection objects, then use the selection objects to create 3D graphical representations. You will also learn how to use the selection objects to produce attribute data for futher analysis with Python data tools, including Pandas DataFrames.  You will use an interactive 2D spike raster plot in the Jupyter notebook window to control a 3D model display in the VMD OpenGL window, and create visualizations and data sets from the interactive selection.



*First, launch VND and load a model...*
* Lauch the VND application (double click icon)
 * (if using command line, enable ZMQ communication by starting VND with ``vnd -zmq`` )
* Load a model:
 **File : Open File** ``vnd-tutorial-files/300_cells/circuit_config.json``
* ...then load a spike time series:
 **File : Add File with Spikes** ``vnd-tutorial-files/300_cells/output/spikes.h5``

*Then, start up a connection from this notebook to the VND session with:*

In [ ]:
import VND


*Now make a selction object using the VND selection langugage...*

In [ ]:
mySel=VND.NeuronSel('(soma x < -15) && (soma y < 0) && (population == internal)')



*Check the status of this object, and see how many neurons are in the selection:*

In [ ]:
mySel

*Make a new graphical representation, which will appear in the VND 3D display and be listed in the VND main tab Representations list:*


In [ ]:
VND.add_rep(sel=mySel, color='red', style='soma', scaling='1.8', resolution="10", material='Opaque', show='True')


*List some attribute values of your selection...*

In [ ]:
mySel.get(['x','node', 'model_name'])


*Display available attributes for currently loaded model...*

In [ ]:
VND.list_attributes()


*Use  simple .get queries, connected to selections and GUI representations, to populate Pandas DataFrames...*

In [ ]:
internalSel=VND.NeuronSel('population == internal ')


In [ ]:
internalSel

In [ ]:
attribs=VND.list_attributes()

In [ ]:
import pandas as pd
import numpy as np

*Combine column data and column titles for the attributes to make a Pandas DataFrame:*


In [ ]:
internal_nodes_df=pd.DataFrame(internalSel.get(attribs),columns=attribs)


*Choose a random 10 perent of the excitatory cells:*


In [ ]:
exc_cell_ids = internal_nodes_df[internal_nodes_df['ei'] == 'e']['node_id'].values
exc_cell_ids = np.random.choice(exc_cell_ids, size=int(len(exc_cell_ids)*.1), replace=False)


In [ ]:
exc_cell_ids

*Choose a random 10 perent of the inhibitory cells:*

In [ ]:
inh_cell_ids = internal_nodes_df[internal_nodes_df['ei'] == 'i']['node_id'].values
inh_cell_ids = np.random.choice(inh_cell_ids, size=int(len(inh_cell_ids)*.1), replace=False)


In [ ]:
inhSel=VND.NeuronSel(f'(node_id == {" ".join(inh_cell_ids)}) && (population == internal)')

In [ ]:
inhSel

In [ ]:
excSel=VND.NeuronSel(f'(node_id == {" ".join(exc_cell_ids)}) && (population == internal)')

In [ ]:
excSel

In [ ]:
VND.add_rep(sel=excSel, color='pink', style='soma', scaling='2.0', material='Opaque', resolution='15',show='True')


In [ ]:
VND.add_rep(sel=inhSel, color='iceblue', style='morphology', scaling='1.6', material='Opaque', show='True')


*Work with spike (action potential) event data.*

*In VND: set up spike event viewing/analysis by first using the VND Activity tab to animate any current selection: choose* **Select Population: internal**, choose a selection from* **Select existing selection:**, then click **Update selection**. *Drag the slider and asjust **time window** control and playback controls see animated spike activity.* 

*Then, create a selection object of the whole population, get the spike event data, and make a DataFrame:*

In [ ]:
internSel=VND.NeuronSel('population == internal')
d = internSel.get_events_dict("internal")
spikes_df = pd.DataFrame({
    'node_id':d['n'],
    'timestamps':d['t']
})
spikes_df

*Find cells with the highest spiking rates:*

In [ ]:
# group df by id and use 'count' function to determine firing rates, 
# sort DataFrame by firing rate and get the top 10
min_ts, max_ts = 0, np.ceil(spikes_df['timestamps'].max())
firing_rates_df = (spikes_df.groupby('node_id').agg('count')/(max_ts - min_ts)*1000.0).reset_index()
firing_rates_df = firing_rates_df.rename(columns={'timestamps': 'firing_rate'})
firing_rates_df = firing_rates_df.sort_values('firing_rate', ascending=False)
fastest_cell_ids = firing_rates_df['node_id'].head(10).values
fastest_cell_ids_seltext=' '.join(map(str, fastest_cell_ids))

In [ ]:
fastest_cell_ids_seltext

In [ ]:
fastestSel=VND.NeuronSel(f'(node_id == {fastest_cell_ids_seltext} ) && (population == internal)')

In [ ]:
fastestSel

In [ ]:
fastestSel.get(['node', 'ei', 'model_name'])


*Animated spike events in VND.*

*Create a representation for the the fastestSel selection; then animate it in VND using the VND Activity tab:*

In [ ]:
#create the rep you will select in the VND GUI in the Activity window
VND.add_rep(sel=fastestSel, color='red', style='soma', scaling='1.8', resolution=6, material='Opaque', show='True')

*After animating the representation in the VND Activity tab, go to the VND Main tab, and hide the represenation for fastestSel that you just made.*

*Now, use selection objects to draw an interactive raster spike plot for the whole model.  In the plot, use the right-margin Lasso tool (and other controls) to examine highlighted events, and see the selections appear in 3D in the VND OpenGL window.  When using the Lasso tool, use shift to add to the dynamic selection, and click on empty background to reset the selection.*

*Note: to see the changing 3D representations more clearly (i.e. less visual clutter), turn off some other representations in the VMD GUI.*

In [ ]:
#panel graph
from bokeh.models import ColumnDataSource, NumeralTickFormatter, LassoSelectTool, HoverTool, TapTool
from bokeh.plotting import figure, show
from bokeh.events import Tap, SelectionGeometry
from bokeh.models.callbacks import CustomJS
import panel as pn
from bokeh.io import reset_output,output_notebook

pn.extension()

plottingSel = VND.NeuronSel('population == internal')
#Make a rep for the  the plottingSel selection; then animate it in the Activity window


dynSel = VND.NeuronSel('! (all)')
dyn_plot_rep = VND.add_rep(sel=dynSel, color='cyan', style='soma', scaling='2.1', resolution="10", material='Transparent', show='True')
dyn_plot_highlight = ""
print (f'dyn_plot_rep is {dyn_plot_rep}')
vndSpikeEventData = plottingSel.get_events_plot_dict("internal")
#now clear 
#reset_output()
#output_notebook()

source = ColumnDataSource(vndSpikeEventData)
selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='VND Spike Graph: Select the events',
             height=500, width=600,
             x_range=(0, max(vndSpikeEventData["x"])), y_range=(0, max(vndSpikeEventData["y"])))

hover_glyph = fig.scatter(source=source, x='x', y='y',
           color='c', size=3, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('t (ms)', '@x'), ('node', '@y'), ('c', '@c'), ('global_id', '@g')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())
fig.add_tools(LassoSelectTool())

def selection_change(attr, old, new):
    #global my_count
    #let data = Object.assign({}, source.data);
    #var kernel = Jupyter.notebook.kernel;    
    #var inds = source.selected.indices;
    global dyn_plot_rep
    global dyn_plot_highlight
    selected_indices = source.selected.indices
    #get gid from the g field
    #the_count = len(selected_indices)
    # [source['g'][i] for i in selected_indices]
    #selection_pane.object = f"Selected {len(selected_indices)} points: {selected_indices}  count: {the_count}"
    #my_count = the_count
    #let indices = Object.assign([], source.selected.indices);
    #data.g =indices.map(i=>data.g[i]);
    #var data_g_str = data.g.toString();
    #var data_g_repl = data_g_str.replace(/,/g," ");
    values_at_indices= [vndSpikeEventData['g'][i] for i in selected_indices]
    values_at_indices_text = ' '.join(str(x) for x in values_at_indices)
    VND.vndSocketReq(f'::neuro::cmd_mod_rep_node_gid_list {dyn_plot_rep} soma cyan Transparent 2.1 10 [list {values_at_indices_text}]')
    dyn_plot_highlight=values_at_indices_text

source.selected.on_change('indices', selection_change)

# Layout and display
# eliminate column? return extending list of gids or node info?
layout = pn.Column(fig)
layout.servable()

*Make a selection from the dynamic plot selection (you can repeat this when you change the graph selection with the Lasso tool etc.)...*

In [ ]:
dynSel = VND.NeuronSel(f'gid == {dyn_plot_highlight}')
dynSel


In [ ]:
dynSel.get(['node_id', 'ei', 'model_name'])

*Make a Pandas DataFrame from the dynSel attribute data...*

In [ ]:
attribs=VND.list_attributes()
dyn_nodes_df=pd.DataFrame(dynSel.get(attribs),columns=attribs)
dyn_nodes_df

*Use the DataFrame to inventory the model names of cells in this selection*

In [ ]:
dyn_nodes_df['model_name'].value_counts()

*Make a graph for the fastest firing neurons only, using the ``fastestSel`` selection...*

*First, check the number of cells in the fastestSel selection object that we made above:*

In [ ]:
fastestSel

*Now make a raster plot graph for only fastestSel, and use the Lasso tool to make selections:*

In [ ]:
#panel graph for fastestSel
from bokeh.models import ColumnDataSource, NumeralTickFormatter, LassoSelectTool, HoverTool, TapTool
from bokeh.plotting import figure, show
from bokeh.events import Tap, SelectionGeometry
from bokeh.models.callbacks import CustomJS
import panel as pn
from bokeh.io import reset_output,output_notebook

pn.extension()

#Make a rep for the  the fastestSel selection; then animate it in the Activity window

dynSel_2 = VND.NeuronSel('! (all)')
dyn_plot_rep_2 = VND.add_rep(sel=dynSel_2, color='magenta', style='soma', scaling='2.1', resolution="10", material='Transparent', show='True')
dyn_plot_highlight_2 = ""
print (f'dyn_plot_rep_2 is {dyn_plot_rep_2}')
vndSpikeEventDataFastest = fastestSel.get_events_plot_dict("internal")
#now clear 
#reset_output()
#output_notebook()

source = ColumnDataSource(vndSpikeEventDataFastest)
selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig2 = figure(title='VND Spike Graph: Select the events',
             height=500, width=600,
             x_range=(0, max(vndSpikeEventDataFastest["x"])), y_range=(0, max(vndSpikeEventDataFastest["y"])))

hover_glyph = fig2.scatter(source=source, x='x', y='y',
           color='c', size=3, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('t (ms)', '@x'), ('node', '@y'), ('c', '@c'), ('global_id', '@g')]

fig2.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig2.add_tools(TapTool())
fig2.add_tools(LassoSelectTool())

def selection_change(attr, old, new):
    #global my_count
    #let data = Object.assign({}, source.data);
    #var kernel = Jupyter.notebook.kernel;    
    #var inds = source.selected.indices;
    global dyn_plot_rep_2
    global dyn_plot_highlight_2
    selected_indices = source.selected.indices
    #get gid from the g field
    #the_count = len(selected_indices)
    # [source['g'][i] for i in selected_indices]
    #selection_pane.object = f"Selected {len(selected_indices)} points: {selected_indices}  count: {the_count}"
    #my_count = the_count
    #let indices = Object.assign([], source.selected.indices);
    #data.g =indices.map(i=>data.g[i]);
    #var data_g_str = data.g.toString();
    #var data_g_repl = data_g_str.replace(/,/g," ");
    values_at_indices= [vndSpikeEventDataFastest['g'][i] for i in selected_indices]
    values_at_indices_text = ' '.join(str(x) for x in values_at_indices)
    VND.vndSocketReq(f'::neuro::cmd_mod_rep_node_gid_list {dyn_plot_rep_2} soma magenta Transparent 2.1 10 [list {values_at_indices_text}]')
    dyn_plot_highlight_2=values_at_indices_text

source.selected.on_change('indices', selection_change)

# Layout and display
# eliminate column? return extending list of gids or node info?
layout = pn.Column(fig2)
layout.servable()